In [1]:
# Import the dependencies
import os
import sys
import time
from collections import Counter
from dotenv import load_dotenv
from functools import reduce
from itertools import combinations
from joblib import Parallel, delayed
from pymongo import MongoClient
import pymongoarrow as pma
from pymongoarrow.api import write
import numba
from numba.typed import List
import numpy as np
from numpy.random import default_rng
import pandas as pd

from sklearn.datasets import make_classification
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import HDBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import MaxAbsScaler

import hvplot.pandas
from matplotlib import pyplot as plt
from ydata_profiling import ProfileReport

# Suppress YData profile report generation warnings - no actual problems to resolve.
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
# Apply latest settings for Pandas
pd.options.mode.copy_on_write = True

## Create a test dataset

In [3]:
# ## Create a test dataset
X, y = make_classification(n_samples=1000, n_features=50, n_informative=5, n_redundant=45, n_repeated=0, n_classes=5, n_clusters_per_class=1, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=42) 

In [4]:
test_df = pd.DataFrame(X)
test_df = test_df.add_prefix('feature_')
test_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,3.079844,4.082231,-0.928285,-1.406834,3.821611,-1.454153,0.558609,-2.833646,1.207253,-2.156476,...,-2.225367,-1.002860,-3.506234,2.659189,1.335290,-0.193650,-2.842733,0.281155,-0.036362,-0.106037
1,-2.168850,-2.940722,-0.120378,1.911458,-4.086751,3.005609,1.061180,-0.546356,-3.393992,1.606306,...,-3.673654,1.082064,2.408369,-0.871287,-1.531216,-1.078653,0.172315,0.802343,3.421586,2.074265
2,-0.118903,-2.292353,-1.772501,2.180867,0.157815,-1.789476,0.766534,0.560211,1.261955,0.183973,...,0.142958,-0.866399,-0.362532,-0.202082,0.837790,-1.247566,-1.536458,-0.201039,-1.277475,1.892091
3,-1.059561,-3.351614,-0.785609,1.608111,-2.891339,-1.140441,1.951887,-0.236173,-0.437770,2.257933,...,-1.997639,0.278740,0.688077,0.335144,-0.791574,-2.209142,-1.045707,-1.063227,0.425318,2.813590
4,2.027416,1.913728,0.206195,-2.121960,1.021316,-3.534001,1.676292,-1.997667,1.832500,0.889809,...,-1.110175,-0.141692,-2.646638,2.944468,-0.104274,-1.446555,-1.741149,-2.128734,-1.182597,0.541159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.468921,6.487245,-0.783022,0.085301,5.309236,2.501753,-0.682770,-3.919702,0.031491,-3.261642,...,-2.955460,-0.274155,-3.475691,1.220751,0.361299,0.269580,-2.155254,2.413773,2.309532,-2.087254
996,0.568388,0.673732,-0.928115,1.489449,0.752600,-2.087146,2.086560,-2.462543,1.801995,1.800496,...,-1.962576,0.328587,-2.421311,1.516172,-1.255831,-3.225189,-2.230410,-1.359061,-0.114012,1.170844
997,0.452091,-1.445143,0.271223,-1.897568,-1.823580,-1.299518,0.661519,0.482176,-0.896208,0.141058,...,-0.875607,-0.393528,0.711648,1.156760,0.846857,0.561399,-0.248216,-0.770103,-0.226960,1.482281
998,4.512252,-0.314382,-4.586837,-0.767961,4.679979,-2.145382,-0.849245,0.728320,-0.187187,-7.886254,...,-2.245725,-4.996025,-1.804474,1.994489,7.835717,3.577882,-5.327821,3.437536,-1.587368,3.600663


## Principal Feature Analysis ##

#### Define functions to select dataset features that provide relevant information for clustering. 
##### Only important features are used to compute clusters from the complete (non-pca) dataset.

In [5]:
# =============================================================================
# Helper function - Custom Mean processing to override limitations of Numba compatibility with Numpy features
@numba.jit(nopython=True)
def custom_mean(arr):
    if arr.ndim == 1:
        return arr.mean()
    elif arr.ndim == 2:
        # For 2D arrays, manually compute the mean of each column.
        means = np.zeros(arr.shape[1])
        for i in range(arr.shape[1]):
            means[i] = arr[:, i].mean()
        return means
    else:
        raise ValueError("Invalid array dimensions for custom_mean")

In [6]:
# =============================================================================
# Helper function - Calinski Harbasz Score Calculation
def calculate_calinski_harbasz(np_array, labels):
    if len(np.unique(labels)) > 1:
        c_h = calinski_harabasz_score(np_array, labels)
        calinski_harbasz = log_scale_value(c_h)
        #print(f'    calinski_harbasz index = {calinski_harbasz}')
        return calinski_harbasz
    else:
        return 0

In [7]:
# =============================================================================
# Helper function - Davies-Bouldin Score Calculation
def calculate_davies_bouldin(np_array, labels):
    if len(np.unique(labels)) > 1:
        davies_bouldin = davies_bouldin_score(np_array, labels)
        #print(f'   davies_bouldin score = {davies_bouldin}')
        return davies_bouldin
    else:
        return 0

In [8]:
# =============================================================================
# Helper function - Silhouette Coefficient Calculation
def calculate_silhouette(np_array, labels):
    if len(np.unique(labels)) > 1:
        silhouette_val = silhouette_score(np_array, labels)
        #print(f' Silhouette Score = {silhouette_val}')
        return silhouette_val
    else:
        return 0

In [9]:
# =============================================================================
# Helper function - Scatter Separability Calculation
def calculate_scatter_separability(np_array, labels):
    """
    Calculates the Scatter Separability (SSC) between clusters in a NumPy array.

    Args:
        np_array: The NumPy array containing the data points.
        labels: The cluster labels for each data point.

    Returns:
        The Scatter Separability score (SSC).
    """
    unique_labels = np.unique(labels)
    n_features = np_array.shape[1]
    overall_mean = custom_mean(np_array)

    S_w = np.zeros((n_features, n_features))
    S_b = np.zeros((n_features, n_features))

    for label in unique_labels:
        X_k = np_array[labels == label]
        mean_k = custom_mean(X_k)
        if mean_k.ndim == 1:
            mean_k = mean_k[:, None]  # Ensure mean_k is a column vector
        n_k = X_k.shape[0]  # Number of samples in current class
        mean_diff = mean_k - overall_mean[:, None]  # Ensure mean_diff is correctly shaped
    
        S_w_k = np.cov(X_k, rowvar=False, bias=True) * (n_k - 1)  # Compute within-class scatter for class k
        S_w += S_w_k
    
        mean_diff = mean_diff.reshape(-1, 1)  # Reshape mean_diff as column vector if not already
        S_b += n_k * np.dot(mean_diff, mean_diff.T)  # Correct outer product computation

    # Handle singular S_w by adding a small identity matrix to ensure invertibility
    if np.linalg.cond(S_w) > 1e10:
        S_w += np.eye(S_w.shape[0]) * 1e-4

    ssc = np.trace(np.linalg.inv(S_w).dot(S_b))
    final_ssc = log_scale_value(ssc)
    #print(f'+ computed SSC - value = {final_ssc} ')
    return final_ssc

In [10]:
# =============================================================================
# Helper function - Performs Logarithmic scaling of cluster quality score metrics that have unbounded positive ranges - Numba acceleration
@numba.jit(nopython=True)
def log_scale_value(value, offset=1):
    """
    Applies a logarithmic scaling to a value.
    
    Parameters:
    - value: The positive metric value to be scaled.
    - offset: A small positive value to avoid log(0) when the metric is zero.
    - scale_max: An upper limit to scale the logarithmic value to, for normalization.
    
    Returns:
    - scaled_value: The logarithmically scaled value, normalized to the range [0, scale_max].
    """
    # Apply logarithmic scaling
    log_scaled_value = np.log(value + offset)
    
    # TBD, normalize the log-scaled value to a specific range, e.g., [0, scale_max]
    # if/when implemented, add ", scale_max=1000" as a function parameter
    
    return log_scaled_value


In [11]:
# =============================================================================
# Helper function - Normalization of criterion values to remove bias due to number of clusters - Numba acceleration
@numba.jit(nopython=True)
def cross_projection_normalization(clustering_medoids, scatter_criteria_score, silhouette_criteria_score, davies_bouldin_score, calinski_harbasz_index):
    n_clusters = len(clustering_medoids)
    projections = np.zeros((n_clusters, n_clusters))

    for j in range(n_clusters):
        for k in range(j + 1, n_clusters):
            medoid_j = clustering_medoids[j]
            medoid_k = clustering_medoids[k]
            distance = np.linalg.norm(medoid_j - medoid_k)
            projections[j][k] = distance
            projections[k][j] = distance

   # Check if all distances are zero
    if np.all(projections == 0):
        #print("CNP - All pairwise distances are zero! Cannot compute Normalized Score.")
        return 0 
    
    # Flatten the array and filter non-zero distances
    flat_projections = projections.ravel()
    non_zero_projections = flat_projections[flat_projections > 0]
    
    if non_zero_projections.size == 0:
        #print("CNP - non_zero_projections is empty. Cannot compute Normalized Score.")
        return 0

    # Calculate mean of non-zero distances
    mean_projection = np.mean(non_zero_projections)

    # Normalizing the criteria scores with the mean of projections
    # Adjusting the formula to consider Davies-Bouldin Score. Recall: For Davies-Bouldin, lower is better.
    # We add 1 to both the numerator and denominator to ensure it doesn't lead to division by zero or negative values.

    # Combined normalization factor incorporates all metrics.
    normalization_denominator = (1 + mean_projection + davies_bouldin_score) 
    normalization_numerator = (1 + scatter_criteria_score + silhouette_criteria_score + calinski_harbasz_index)
    normalized_score = normalization_numerator / normalization_denominator

    return normalized_score

In [12]:
# =============================================================================
# Primary function - calls functions to generate cluster metrics in parallel
def score_subset_clusters(subset_array, np_array, cluster_labels, clustering_medoids):
    # Define tasks to be executed in parallel
    tasks = [delayed(calculate_scatter_separability)(subset_array, cluster_labels),
             delayed(calculate_silhouette)(subset_array, cluster_labels),
             delayed(calculate_davies_bouldin)(subset_array, cluster_labels),
             delayed(calculate_calinski_harbasz)(subset_array, cluster_labels)]
    
    # Execute tasks in parallel and unpack results
    scatter_separability, silhouette_score, davies_bouldin_score, calinski_harbasz_index = Parallel(n_jobs=4)(tasks)
    
    # Pass the results to the normalization function
    normalized_score = cross_projection_normalization(clustering_medoids, scatter_separability, silhouette_score, davies_bouldin_score, calinski_harbasz_index)

    return normalized_score

In [13]:
# =============================================================================
# Primary function - Initializes the clustering algorithm
def initialize_clustering_instance(clustering_algorithm, rng, flag, common_metric):
    if flag == 'reverse': # Use vanilla algoritms for reverse-search feature scoring to ensure a consistent comparison
        if clustering_algorithm == 'kmedoids':
            return KMedoids(n_clusters=2, init='k-medoids++'), 0, 'na', 0, 0
        elif clustering_algorithm == 'hdbscan':
            return HDBSCAN(store_centers="medoid"), 0, 'na', 0, 0
    else:
        if flag == 'common':
            metric = common_metric
        else:
            metrics = ['manhattan', 'euclidean', 'chebyshev', 'canberra', 'hamming']  # Available metrics - pick one
            metric = rng.choice(metrics)  # Randomly select a metric
            
        if clustering_algorithm == 'kmedoids':
            #metrics = ['manhattan', 'euclidean', 'cosine']  # Available metrics - pick one
            k_val = rng.integers(2, 10)
            return KMedoids(n_clusters=k_val, init='k-medoids++', metric=metric), k_val, metric, 0, 0
        elif clustering_algorithm == 'hdbscan':
            min_cluster_size = rng.choice([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150])
            min_samples = min_cluster_size + rng.choice([10, 20, 30, 40, 50, 60, 70]) # Higher values force conservative clustering
            return HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric=metric, cluster_selection_method='eom', store_centers="medoid"), 0, metric, min_cluster_size, min_samples
        else:
            raise ValueError("Unsupported clustering algorithm")

In [14]:
# =============================================================================
# Primary function - Evaluates the subset_array
def evaluate_subset(clustering_algorithm, subset_array, np_array, rng, flag, common_metric):
    clustering_instance, k_val, metric, min_cluster_size, min_samples = initialize_clustering_instance(clustering_algorithm, rng, flag, common_metric)
    current_labels = clustering_instance.fit_predict(subset_array)
    clustering_medoids = getattr(clustering_instance, 'cluster_centers_', getattr(clustering_instance, 'medoids_', None))
    return score_subset_clusters(subset_array, np_array, current_labels, clustering_medoids), k_val, metric, min_cluster_size, min_samples

In [15]:
# =============================================================================
# Primary function - Creates a starter_set from the available_indices
def select_starter_set(available_indices, interim_features, starter_set_size, rng):
    remaining_indices = list(available_indices - interim_features)
    if len(remaining_indices) <= starter_set_size: # Check if there are enough indices to create a full starter_set
        return rng.shuffle(remaining_indices), 0
    else:
        return rng.choice(remaining_indices, starter_set_size, replace=False), 1

In [16]:
# =============================================================================
# Primary function - Adds a new score and the clustering parameters to the collection
def update_best_scores(best_scores, score, feature, k_val, metric, cluster_size, num_samples, interim_features, available_indices):
    best_scores['k_val'].append(k_val) # K-Medoids - Track the target k
    best_scores['cluster_size'].append(cluster_size) # HDBSCAN - Track the size of the clusters
    best_scores['num_samples'].append(num_samples)  # HDBSCAN - Track the number of samples per cluster
    best_scores['metric'].append(metric)  # K-Medoids - Track the metric
    interim_features.add(feature)
    available_indices.remove(feature)
    return available_indices, interim_features, best_scores

In [17]:
# =============================================================================
# Primary function - Score the interim_features as they are removed - determine whether any ore not needed.
def evaluate_feature_removal(np_array, clustering_algorithm, current_features, feature_to_remove, rng, flag, common_metric):
    # Remove the specified feature from the current feature set
    modified_features = [f for f in current_features if f != feature_to_remove]
    
    # Subset the np_array to include only the modified feature set
    subset_array = np_array[:, modified_features]
    
    # Evaluate the clustering with the modified feature set
    score, _, _, _, _, = evaluate_subset(clustering_algorithm, subset_array, np_array, rng, flag, common_metric)
    
    return score

In [18]:
# =============================================================================
# Primary function - Evaluate the scored interim_features - remove unneeded if below criteria.
def perform_reverse_search(np_array, clustering_algorithm, interim_features, global_best_score, rng):
    refined_features = set(interim_features)
    feature_removed = True

    print(' --------------- Beginning Reverse-searching to refine the interim_features ... --------------- ')

    while feature_removed:
        if len(refined_features) == 5:
            print(' --------------- Need to collect more candidate features - we have 5 features left.')
            return refined_features, global_best_score
        else:
            test_scores = []  # Store feature removal scores for comparison
    
            # Evaluate feature removal in parallel
            test_scores = Parallel(n_jobs=10)(
                delayed(evaluate_feature_removal)(np_array, clustering_algorithm, list(refined_features), feature, rng, 'reverse', 'na')
                for feature in refined_features
            )
            test_scores = list(zip(refined_features, test_scores))  # Combine feature indices with their scores
    
            candidate_removal, candidate_score = max(test_scores, key=lambda x: x[1])  # Use the key argument of the max() function to specify that we want to find the maximum based on the second element of each tuple in test_scores
    
            if candidate_score > global_best_score:  # The score improves with the feature removed and is better than the local best score
                refined_features.remove(candidate_removal)
                global_best_score = candidate_score  # Update the local best score
                print(f' --------------- Feature {candidate_removal} was removed from the refined_features. Current removal score is {global_best_score:.4f}')
                feature_removed = True
            else:
                print(' --------------- No feature met the removal criteria ')
                feature_removed = False
    
    return refined_features, global_best_score

In [19]:
# =============================================================================
# Primary function - Evaluate the available indices - iterate through the collection to identify those that improve scores by removal.
def perform_reverse_index_removal(np_array, clustering_algorithm, available_indices, global_best_score, rng):
    indices_removed = True  # Flag to track if any features were removed in the last pass

    print(' --------------- Starting Reverse Index Removal --------------- ')
    while indices_removed: # remove as many as possible for each run...
        indices_removed = False  # Reset the flag for the current pass
        if len(available_indices) == 10:
            print(' --------------- No index removal possible.')
            return available_indices, global_best_score
        else:
            test_scores = []  # Store feature removal scores for comparison

            # Evaluate feature removal in parallel
            test_scores = Parallel(n_jobs=10)(
                delayed(evaluate_feature_removal)(np_array, clustering_algorithm, list(available_indices), feature, rng, 'reverse', 'na')
                for feature in available_indices
            )
            test_scores = list(zip(available_indices, test_scores))  # Combine feature indices with their scores

            candidate_removal, candidate_score = max(test_scores, key=lambda x: x[1])  # Use the key argument of the max() function to specify that we want to find the maximum based on the second element of each tuple in test_scores

            if candidate_score > (global_best_score + 0.05):  # The score improves with the feature removed and is incrementally better than the global removal score
                available_indices.remove(candidate_removal)
                indices_removed = True  # Indicate that an index was removed in this pass
                global_best_score = candidate_score  # Update the global removal score
                print(f' --------------- Index {candidate_removal} was removed. The current removal score is {global_best_score:.4f}')

    print(f' --------------- Reverse Index Removal completed ---- {len(available_indices)} available_indices remaining. ')
    return available_indices, global_best_score

In [20]:
# =============================================================================
# Helper function - Shuffles the contents of the feature array during combination evaluation
def shuffle_array(array, rng):
    # Shuffle the array along the first axis (rows)
    shuffled_array = array.copy()
    rng.shuffle(shuffled_array, axis=0)
    return shuffled_array

In [21]:
# =============================================================================
# Primary function - Evaluates new features alongside the initial collection of important features
def evaluate_combinations(np_array, clustering_algorithm, available_indices, refined_features, rng, best_scores, global_best_score):
    iteration = 0

    print(' *************** Begin combination evaluation ...  *************** ')

    # Identify the most common measurement metric and use that for the combination evaluations
    metric_counter = Counter(best_scores['metric'])
    common_metric = metric_counter.most_common(1)[0][0]  # Return the most common metric directly
    print(f' ***************  Most common metric: {common_metric}')
        
    while True:  # Simulate a do-while loop
        iteration += 1

        # Create a list of features to evaluate
        features_to_evaluate = list(available_indices - refined_features)

        # Evaluate combination and features in parallel
        evaluation_results = Parallel(n_jobs=10)(
            delayed(evaluate_subset)(clustering_algorithm, shuffle_array(np_array[:, list(refined_features) + [feature]], rng), np_array, rng, 'na', common_metric)
            for feature in features_to_evaluate
        )

        for idx, (normalized_score, k_val, metric, cluster_size, num_samples) in enumerate(evaluation_results):
            feature = features_to_evaluate[idx]

            # Update best combination if necessary
            if normalized_score > global_best_score:
                global_best_score = normalized_score
                best_feature = feature
                best_clust = cluster_size
                best_num_samps = num_samples
                best_k = k_val
                best_metric = metric
                iteration = 0  # Reset the counter

                # If a better combination was found, add its feature to refined_features
                available_indices, refined_features, best_scores = update_best_scores(best_scores, global_best_score, best_feature, best_k, best_metric, best_clust, best_num_samps, refined_features, available_indices)
                print(f' *************** Feature Number {best_feature} added to refined_features with score = {global_best_score:.4f} - {len(refined_features)} refined_features found so far...')

        if iteration % 20 == 0:
            print(f' *************** Combination Evaluation Iteration {iteration}')
        if (iteration % 205 ==0): # Combination feature searching has stalled for some other reason - exit
            break
        
    print(f' *************** Processing completed - Total number of identified features = {len(refined_features)}')

    return available_indices, refined_features, best_scores, global_best_score

In [22]:
# =============================================================================
# Primary function - Select candidate features from the available_indices via starter_sets
def select_candidate_features_via_starter_set(np_array, clustering_algorithm, interim_features, available_indices, best_scores, global_best_score, big_score_jump, rng):
    ss_flag = 1  # Initalize the local variable in case this function is entered and the processing is skipped
    starter_set_size = 10
    k_val, min_cluster_size, min_samples, cluster_size, num_samples, iteration = 0, 0, 0, 0, 0, 0

    print(' +++++++++++++++ Begin selecting candidate features via starter_sets ... +++++++++++++++ ')
         
    while True:  # Simulate a do-while loop
        starter_set, ss_flag = select_starter_set(available_indices, interim_features, starter_set_size, rng)

        # Create a list of features to evaluate
        features_to_evaluate = list(available_indices - interim_features - set(starter_set))

        # Evaluate features in parallel
        evaluation_results = Parallel(n_jobs=10)(
            delayed(evaluate_subset)(clustering_algorithm, np_array[:, list(starter_set) + [feature]], np_array, rng, 'na', 'na')
            for feature in features_to_evaluate
        )

        for idx, (normalized_score, k_val, metric, cluster_size, num_samples) in enumerate(evaluation_results):
            feature = features_to_evaluate[idx]

            # Update best feature if necessary
            if normalized_score > global_best_score:
                if (normalized_score - global_best_score) > 0.7:
                    big_score_jump = True
                    interim_features.update(starter_set)
                    available_indices.difference_update(starter_set)
                    print(' +*+*+*+*+*+*  Detected a big_score_jump - Added the entire starter_set to the interim_features *+*+*+*+*+*+')
                global_best_score = normalized_score
                best_new_feature = feature
                best_clust = cluster_size
                best_num_samps = num_samples
                best_k = k_val
                best_metric = metric
                
                available_indices, interim_features, best_scores = update_best_scores(best_scores, global_best_score, best_new_feature, best_k, best_metric, best_clust, best_num_samps, interim_features, available_indices)
                print(f' +++++++++++++++ Found a new interim feature: {best_new_feature} - with score = {global_best_score:.4f} - {len(interim_features)} found so far...')
                iteration = 0  # Reset the flag
        
        if ss_flag == 0:
            print(' +++++++++++++++  Finished processing all available features ... ')
            break
        else:
            iteration += 1
            if iteration % 40 ==0:
                print(f' +++++++++++++++ Starter_set Filling Iteration {iteration}')
            if (iteration % 205 ==0): # Starter_set feature searching has stalled for some other reason - exit
                print(f' +++++++++++++++ Exiting Starter_set Filling - {len(interim_features)} Interim Features found so far...')
                break

    return available_indices, interim_features, best_scores, global_best_score, ss_flag, big_score_jump

In [26]:
# =============================================================================
# Main function - Orchestrates the analysis
def optimal_feature_clusters(np_array, clustering_algorithm):
    rng = default_rng()  # Use numpy's random number generation.
    n_features = np_array.shape[1]
    available_indices = set(range(n_features))
    interim_features = set()
    best_scores = {'k_val': [], 'cluster_size': [], 'num_samples': [], 'metric': []}
    best_score = 0.2 # Starter score for first pass into select_candidate_features_via_starter_set
    big_score_jump = False  #Prepare the flag for locating a very important feature

    # Start the processing by looking for features that negatively impact scoring - remove them from the set (reducing set size increases processing speed).
    print(' =============== Start processing ... ')
    available_indices, global_best_score = perform_reverse_index_removal(np_array, clustering_algorithm, available_indices, best_score, rng)
    available_indices, refined_features, best_scores, global_best_score, ss_flag, big_score_jump = select_candidate_features_via_starter_set(np_array, clustering_algorithm, interim_features, available_indices, best_scores, best_score, big_score_jump, rng)
    updated = True
    
    while updated and (ss_flag == 1):
        # Use the current candidates to locate any new important features
        available_indices, refined_features, best_scores, global_best_score = evaluate_combinations(np_array, clustering_algorithm, available_indices, refined_features, rng, best_scores, global_best_score)

        # Test the current candidates to remove any that appear to be restricting the scores
        refined_features, global_best_score = perform_reverse_search(np_array, clustering_algorithm, refined_features, global_best_score, rng)
        refined_features_after_reverse_search = len(refined_features)  # Collect the current number of refined features
        
        if big_score_jump:
            # Try to remove any unimportant indices
            available_indices, global_best_score = perform_reverse_index_removal(np_array, clustering_algorithm, available_indices, global_best_score, rng)
            # Make a limited attempt to locate any more features with the starter sets
            available_indices, refined_features, best_scores, global_best_score, ss_flag, global_removal_score, big_score_jump = select_candidate_features_via_starter_set(np_array, clustering_algorithm, refined_features, available_indices, best_scores, global_best_score, big_score_jump, rng)

            refined_features, global_best_score = perform_reverse_search(np_array, clustering_algorithm, refined_features, global_best_score, rng)
            
            if len(refined_features) > refined_features_after_reverse_search:  # Test if we have collected more features through the processing
                updated = True
                print(' =============== Moving to next iteration... ') # Looks like we got past the big score jump
                big_score_jump = False # Reset the flag
                continue
            else:
                # Collect the current state of the candidates
                current_state = len(refined_features)
                # Try to work with the current candidates and explore combinations
                available_indices, refined_features, best_scores, global_best_score = evaluate_combinations(np_array, clustering_algorithm, available_indices, refined_features, rng, best_scores, global_best_score)

                if len(refined_features) > current_state:  # We made progress, so keep working...
                    updated = True
                    print(' =============== Moving to next iteration... ') # Looks like we got past the big score jump
                    big_score_jump = False # Reset the flag
                    continue
                else:
                    print(' =============== Processing completed. ') # Still blocked by the big score jump
                    break
        else: # Continue processing features
            updated = False

            # Identify any more of the available_indices that negatively affect scoring - remove them
            available_indices, global_best_score = perform_reverse_index_removal(np_array, clustering_algorithm, available_indices, global_best_score, rng)

            available_indices, refined_features, best_scores, global_best_score, ss_flag, big_score_jump = select_candidate_features_via_starter_set(np_array, clustering_algorithm, refined_features, available_indices, best_scores, global_best_score, big_score_jump, rng)

            refined_features, global_best_score = perform_reverse_search(np_array, clustering_algorithm, refined_features, global_best_score, rng)
            
            available_indices, refined_features, best_scores, global_best_score = evaluate_combinations(np_array, clustering_algorithm, available_indices, refined_features, rng, best_scores, global_best_score)

            refined_features, global_best_score = perform_reverse_search(np_array, clustering_algorithm, refined_features, global_best_score, rng)
            
            if len(refined_features) != refined_features_after_reverse_search:  # Test if the number of collected features has changed through the processing
                updated = True
                print(' =============== Moving to next iteration... ')
            else:
                print(' =============== Processing completed. ')

    return best_scores['k_val'], best_scores['metric'], refined_features, best_scores['cluster_size'], best_scores['num_samples'],

### Perform PFA

#### KMedoids

In [27]:
# ^^^^^^^^^^^^^^^^^KMedoids Run 1 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

best_kmedoid_features_run1 = set()

complete1_df = test_df.copy()
complete1_np = complete1_df.to_numpy()

# Run the experiment using the complete (non-pca) dataframe and identify the clustering algorithm by name.
best_k_vals_run1, best_kmetric_run1, best_kmedoid_features_run1, na1, na2 = optimal_feature_clusters(complete1_np, 'kmedoids')

# Stop timing
stop = time.perf_counter()

print(f' ^^^ RUN #1 --- PFA KMedoids Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best k values = {sorted(best_k_vals_run1)}')
print(f' Best metric values = {sorted(best_kmetric_run1)}')
print(f' best features = {sorted(best_kmedoid_features_run1)}')

 =============== Start processing ... 
 --------------- Starting Reverse Index Removal --------------- 
 --------------- Index 43 was removed. The current removal score is 0.6297
 --------------- Reverse Index Removal completed ---- 49 available_indices remaining. 
 +++++++++++++++ Begin selecting candidate features via starter_sets ... +++++++++++++++ 
 +++++++++++++++ Found a new interim feature: 7 - with score = 0.3604 - 1 found so far...
 +++++++++++++++ Found a new interim feature: 24 - with score = 0.3728 - 2 found so far...
 +++++++++++++++ Found a new interim feature: 49 - with score = 0.4097 - 3 found so far...
 +++++++++++++++ Found a new interim feature: 1 - with score = 0.9499 - 4 found so far...
 +++++++++++++++ Found a new interim feature: 2 - with score = 0.9674 - 5 found so far...
 +++++++++++++++ Found a new interim feature: 8 - with score = 0.9688 - 6 found so far...
 +++++++++++++++ Found a new interim feature: 16 - with score = 1.0120 - 7 found so far...
 ++++++++++

In [ ]:
# ^^^^^^^^^^^^^^^^^KMedoids Run 2 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

best_kmedoid_features_run2 = set()

complete2_df = test_df.copy()
complete2_np = complete2_df.to_numpy()

# Run the experiment using the complete (non-pca) dataframe and identify the clustering algorithm by name.
best_k_vals_run2, best_kmetric_run2, best_kmedoid_features_run2, na1, na2 = optimal_feature_clusters(complete2_np, 'kmedoids')

# Stop timing
stop = time.perf_counter()

print(f' ^^^ RUN #2 --- PFA KMedoids Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best k values = {sorted(best_k_vals_run2)}')
print(f' Best metric values = {sorted(best_kmetric_run2)}')
print(f' best features = {sorted(best_kmedoid_features_run2)}')

In [ ]:
# ^^^^^^^^^^^^^^^^^KMedoids Run 3 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

best_kmedoid_features_run3 = set()

complete3_df = test_df.copy()
complete3_np = complete3_df.to_numpy()

# Run the experiment using the complete (non-pca) dataframe and identify the clustering algorithm by name.
best_k_vals_run3, best_kmetric_run3, best_kmedoid_features_run3, na1, na2 = optimal_feature_clusters(complete3_np, 'kmedoids')

# Stop timing
stop = time.perf_counter()

print(f' ^^^ RUN #3 --- PFA KMedoids Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best k values = {sorted(best_k_vals_run3)}')
print(f' Best metric values = {sorted(best_kmetric_run3)}')
print(f' best features = {sorted(best_kmedoid_features_run3)}')

In [ ]:
# ^^^^^^^^^^^^^^^^^KMedoids Run 4 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

best_kmedoid_features_run4 = set()

complete4_df = test_df.copy()
complete4_np = complete4_df.to_numpy()

# Run the experiment using the complete (non-pca) dataframe and identify the clustering algorithm by name.
best_k_vals_run4, best_kmetric_run4, best_kmedoid_features_run4, na1, na2 = optimal_feature_clusters(complete4_np, 'kmedoids')

# Stop timing
stop = time.perf_counter()

print(f' ^^^ RUN #4 --- PFA KMedoids Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best k values = {sorted(best_k_vals_run4)}')
print(f' Best metric values = {sorted(best_kmetric_run4)}')
print(f' best features = {sorted(best_kmedoid_features_run4)}')

In [ ]:
# ^^^^^^^^^^^^^^^^^KMedoids Run 5 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

best_kmedoid_features_run5 = set()

complete5_df = test_df.copy()
complete5_np = complete5_df.to_numpy()

# Run the experiment using the complete (non-pca) dataframe and identify the clustering algorithm by name.
best_k_vals_run5, best_kmetric_run5, best_kmedoid_features_run5, na1, na2 = optimal_feature_clusters(complete5_np, 'kmedoids')

# Stop timing
stop = time.perf_counter()

print(f' ^^^ RUN #5 --- PFA KMedoids Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best k values = {sorted(best_k_vals_run5)}')
print(f' Best metric values = {sorted(best_kmetric_run5)}')
print(f' best features = {sorted(best_kmedoid_features_run5)}')

### Process the collected results

In [ ]:
# Identify common k-value and features in the KMedoids output
# Combine the run results
all_k_values = best_k_vals_run1 + best_k_vals_run2 + best_k_vals_run3 + best_k_vals_run4 + best_k_vals_run5
all_kmetric_values = best_kmetric_run1 + best_kmetric_run2 + best_kmetric_run3 + best_kmetric_run4 + best_kmetric_run5
all_features = [best_kmedoid_features_run1, best_kmedoid_features_run2, best_kmedoid_features_run3, best_kmedoid_features_run4, best_kmedoid_features_run5]

# Count the frequency of the values and pick the maximum in the event of a tie
k_counter = Counter(all_k_values)
most_common_k_values = k_counter.most_common()  # This gives a list of (k, count) pairs
max_count = most_common_k_values[0][1]  # The count of the most frequent k
# Filter for ties: get all k values with the count equal to max_count
ties = [k for k, count in most_common_k_values if count == max_count]
# Select the largest k from the ties
selected_k = max(ties)  # Select the largest k from the ties

kmetric_counter = Counter(all_kmetric_values)
most_common_kmetric = kmetric_counter.most_common(1)[0][0]  # Return the most common metric directly
print(f'Most common metric: {most_common_kmetric}')

# For features, use set operations to find common and all selected features
final_kmedoid_common_features = set.intersection(*all_features)
final_kmedoid_combined_features = set.union(*all_features)

In [ ]:
print(sorted(final_kmedoid_common_features))

In [ ]:
print(sorted(final_kmedoid_combined_features))

### Perform clustering with the reduced feature set

In [ ]:
# Use the selected features for the final KMedoids clustering

# Add a test for whether there are any common results
if (final_kmedoid_common_features == set()) or (len(final_kmedoid_common_features) <= 5): # no common features OR not enough for processing
    print(' No common features - skipping ahead... ')
else:
    # Create a DataFrame for the best common features
    kmedoids_common_df = test_df.copy()
    kmedoids_reduced_common_features_df = kmedoids_common_df.iloc[:,sorted(final_kmedoid_common_features)]
    
    # Perform clustering on the final set of common features with the common k-value
    kmedoids_final_common_model = KMedoids(n_clusters=selected_k, init='k-medoids++', metric=most_common_kmetric, random_state=42)
    kmedoids_final_common_labels = kmedoids_final_common_model.fit_predict(kmedoids_reduced_common_features_df)
    kmedoids_final_common_cluster_centers = kmedoids_final_common_model.cluster_centers_
    
    # Create the dataframes for visualization
    kmedoids_final_viz_common_features_df = test_df.copy()
    kmedoids_final_reduced_common_features_df = kmedoids_final_viz_common_features_df.iloc[:,sorted(final_kmedoid_common_features)]
    kmedoids_final_reduced_common_features_df['KMedoids Clusters'] = kmedoids_final_common_labels
    
    kmedoids_final_COMMON_complete_features_df = test_df.copy()
    kmedoids_final_COMMON_complete_features_df['KMedoids Clusters'] = kmedoids_final_common_labels

# ------------------------------------------------------------------------------------------
# Create a DataFrame for the best combined features
kmedoids_combined_df = test_df.copy()
kmedoids_reduced_combined_features_df = kmedoids_combined_df.iloc[:,sorted(final_kmedoid_combined_features)]

# Perform clustering on the final set of combined features with the common k-value
kmedoids_final_combined_model = KMedoids(n_clusters=selected_k, init='k-medoids++', metric=most_common_kmetric, random_state=42)
kmedoids_final_combined_labels = kmedoids_final_combined_model.fit_predict(kmedoids_reduced_combined_features_df)
kmedoids_final_combined_cluster_centers = kmedoids_final_combined_model.cluster_centers_

# Create the dataframes for visualization
kmedoids_final_viz_combined_features_df = test_df.copy()
kmedoids_final_reduced_combined_features_df = kmedoids_final_viz_combined_features_df.iloc[:,sorted(final_kmedoid_combined_features)]
kmedoids_final_reduced_combined_features_df['KMedoids Clusters'] = kmedoids_final_combined_labels

kmedoids_final_COMBINED_complete_features_df = test_df.copy()
kmedoids_final_COMBINED_complete_features_df['KMedoids Clusters'] = kmedoids_final_combined_labels

### Generate final K-Medoids Reports

In [ ]:
# Define a function that generates a profile report and saves it to a file
def generate_report(df, config_file, output_file):
    profile = ProfileReport(df, config_file=config_file)
    profile.to_file(output_file)
    print(f"Report {output_file} generated.")

In [ ]:
# Start timing
start = time.perf_counter()

# Create YData reports to explore the KMedoids feature relationships
# DataFrames and configuration for the reports

# Add a test for whether there are any common results
if (final_kmedoid_common_features == set()) or (len(final_kmedoid_common_features) <= 5): # no common features found
    print(' No common features')
    now = str(time.time_ns()) # Create a timestamp for unique filename sets
    reports_info = [
        {
            'df': kmedoids_final_reduced_combined_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'KMedoids_Final_REDUCED_COMBINED-Features_Report-' + now + '.html'
        },
        {
            'df': kmedoids_final_COMBINED_complete_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'KMedoids_Final_COMPLETE_COMBINED-Features_Report-' + now + '.html'
        }
    ]
else:
    now = str(time.time_ns()) # Create a timestamp for unique filename sets
    reports_info = [
        {
            'df': kmedoids_final_reduced_common_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'KMedoids_Final_REDUCED_COMMON-Features_Report-' + now + '.html'
        },
        {
            'df': kmedoids_final_COMMON_complete_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'KMedoids_Final_COMPLETE_COMMON-Features_Report-' + now + '.html'
        },
        {
            'df': kmedoids_final_reduced_combined_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'KMedoids_Final_REDUCED_COMBINED-Features_Report-' + now + '.html'
        },
        {
            'df': kmedoids_final_COMBINED_complete_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'KMedoids_Final_COMPLETE_COMBINED-Features_Report-' + now + '.html'
        }
    ]

# Use joblib to run the report generations in parallel
# n_jobs=-1 uses all available CPUs
Parallel(n_jobs=4)(delayed(generate_report)(
    info['df'], info['config_file'], info['output_file']) for info in reports_info)

# Stop timing
stop = time.perf_counter()

print(f' ^^^ Final KMedoids Clustering Report building in {stop - start:0.4f} seconds ^^^ ')

#### HDBSCAN

In [25]:
# ^^^^^^^^^^^^^^^^^HDBSCAN Run 1 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

complete11_df = test_df.copy()
complete11_np = complete11_df.to_numpy()

best_hdbscan_features_run1 = set()

# Run the experiment using the complete (non-pca) dataframe
na1, best_hmetric_run1, best_hdbscan_features_run1, best_cluster_size_run1, best_num_samples_run1 = optimal_feature_clusters(complete11_np, 'hdbscan')

# Stop timing
stop = time.perf_counter()

print(f' ^^^RUN #1 --- PFA HDBSCAN Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best metric values = {sorted(best_hmetric_run1)}')
print(f' best features 1 = {sorted(best_hdbscan_features_run1)}')

 =============== Start processing ... 
 --------------- Starting Reverse Index Removal --------------- 
 --------------- Reverse Index Removal completed ---- 50 available_indices remaining. 
 +++++++++++++++ Begin selecting candidate features via starter_sets ... +++++++++++++++ 
 +++++++++++++++ Found a new interim feature: 0 - with score = 0.4716 - 1 found so far...
 +++++++++++++++ Found a new interim feature: 7 - with score = 0.4816 - 2 found so far...
 +++++++++++++++ Found a new interim feature: 13 - with score = 0.5194 - 3 found so far...
 +++++++++++++++ Found a new interim feature: 16 - with score = 0.5460 - 4 found so far...
 +++++++++++++++ Found a new interim feature: 3 - with score = 0.5482 - 5 found so far...
 +++++++++++++++ Found a new interim feature: 6 - with score = 0.5782 - 6 found so far...
 +++++++++++++++ Found a new interim feature: 48 - with score = 0.5942 - 7 found so far...
 +++++++++++++++ Starter_set Filling Iteration 40
 +++++++++++++++ Starter_set Filling

In [28]:
# ^^^^^^^^^^^^^^^^^HDBSCAN Run 2 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

complete12_df = test_df.copy()
complete12_np = complete12_df.to_numpy()

best_hdbscan_features_run2 = set()

# Run the experiment using the complete (non-pca) dataframe
na1, best_hmetric_run2, best_hdbscan_features_run2, best_cluster_size_run2, best_num_samples_run2 = optimal_feature_clusters(complete12_np, 'hdbscan')

# Stop timing
stop = time.perf_counter()

print(f' ^^^RUN #2 --- PFA HDBSCAN Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best metric values = {sorted(best_hmetric_run2)}')
print(f' best features 2 = {sorted(best_hdbscan_features_run2)}')

 =============== Start processing ... 
 --------------- Starting Reverse Index Removal --------------- 
 --------------- Reverse Index Removal completed ---- 50 available_indices remaining. 
 +++++++++++++++ Begin selecting candidate features via starter_sets ... +++++++++++++++ 
 +++++++++++++++ Found a new interim feature: 46 - with score = 0.3617 - 1 found so far...
 +++++++++++++++ Found a new interim feature: 0 - with score = 0.4761 - 2 found so far...
 +++++++++++++++ Found a new interim feature: 1 - with score = 0.4919 - 3 found so far...
 +++++++++++++++ Found a new interim feature: 5 - with score = 0.5253 - 4 found so far...
 +++++++++++++++ Found a new interim feature: 22 - with score = 0.5900 - 5 found so far...
 +++++++++++++++ Starter_set Filling Iteration 40
 +++++++++++++++ Starter_set Filling Iteration 80
 +++++++++++++++ Found a new interim feature: 31 - with score = 0.6126 - 6 found so far...
 +++++++++++++++ Starter_set Filling Iteration 40
 +++++++++++++++ Starter_s

In [ ]:
# ^^^^^^^^^^^^^^^^^HDBSCAN Run 3 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

complete13_df = test_df.copy()
complete13_np = complete13_df.to_numpy()

best_hdbscan_features_run3 = set()

# Run the experiment using the complete (non-pca) dataframe
na1, best_hmetric_run3, best_hdbscan_features_run3, best_cluster_size_run3, best_num_samples_run3 = optimal_feature_clusters(complete13_np, 'hdbscan')

# Stop timing
stop = time.perf_counter()

print(f' ^^^RUN #3 --- PFA HDBSCAN Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best metric values = {sorted(best_hmetric_run3)}')
print(f' best features 3  = {sorted(best_hdbscan_features_run3)}')

In [ ]:
# ^^^^^^^^^^^^^^^^^HDBSCAN Run 4 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

complete14_df = test_df.copy()
complete14_np = complete14_df.to_numpy()

best_hdbscan_features_run4 = set()

# Run the experiment using the complete (non-pca) dataframe
na1, best_hmetric_run4, best_hdbscan_features_run4, best_cluster_size_run4, best_num_samples_run4 = optimal_feature_clusters(complete14_np, 'hdbscan')

# Stop timing
stop = time.perf_counter()

print(f' ^^^RUN #4 --- PFA HDBSCAN Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best metric values = {sorted(best_hmetric_run4)}')
print(f' best features 4  = {sorted(best_hdbscan_features_run4)}')

In [ ]:
# ^^^^^^^^^^^^^^^^^HDBSCAN Run 5 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Start timing
start = time.perf_counter()

complete15_df = test_df.copy()
complete15_np = complete15_df.to_numpy()

best_hdbscan_features_run5 = set()

# Run the experiment using the complete (non-pca) dataframe
na1, best_hmetric_run5, best_hdbscan_features_run5, best_cluster_size_run5, best_num_samples_run5 = optimal_feature_clusters(complete15_np, 'hdbscan')

# Stop timing
stop = time.perf_counter()

print(f' ^^^RUN #5 --- PFA HDBSCAN Clustering Execution in {stop - start:0.4f} seconds ^^^ ')
print(f' Best metric values = {sorted(best_hmetric_run5)}')
print(f' best features 5  = {sorted(best_hdbscan_features_run5)}')

### Process the collected results

In [ ]:
# Identify common cluster_size value and features in the HDBSCAN output
# Combine the run results
all_hmetric_values = best_hmetric_run1 + best_hmetric_run2 + best_hmetric_run3 + best_hmetric_run4 + best_hmetric_run5
all_cluster_sizes = best_cluster_size_run1 + best_cluster_size_run2 + best_cluster_size_run3 + best_cluster_size_run4 + best_cluster_size_run5
all_num_samples = best_num_samples_run1 + best_num_samples_run2 + best_num_samples_run3 + best_num_samples_run4 + best_num_samples_run5
all_features = [best_hdbscan_features_run1, best_hdbscan_features_run2, best_hdbscan_features_run3, best_hdbscan_features_run4, best_hdbscan_features_run5]

# Count the frequency of the metric values and pick the most common
hmetric_counter = Counter(all_hmetric_values)
most_common_hmetric = hmetric_counter.most_common(1)[0][0]  # Return the most common metric directly
print(f'Most common metric: {most_common_hmetric}')

# Count the frequency of the values and pick the maximum in the event of a tie
cluster_size_counter = Counter(all_cluster_sizes)
most_common_cluster_size_vals = cluster_size_counter.most_common()    # This gives a list of (cluster_size, count) pairs
max_count_c_s = most_common_cluster_size_vals[0][1]  # The count of the most frequent cluster_size
cluster_ties = [c_size for c_size, count in most_common_cluster_size_vals if count == max_count_c_s]
selected_cluster_size = max(cluster_ties) # We want the largest cluster_size in case of a tie

# Count the frequency of the values and pick the maximum in the event of a tie
sample_size_counter = Counter(all_num_samples)
most_common_sample_size_val = sample_size_counter.most_common()   # This gives a list of (sample_size, count) pairs
max_count_sample_size = most_common_sample_size_val[0][1]  # The count of the most frequent sample_size
sample_ties = [s_num for s_num, count in most_common_sample_size_val if count == max_count_sample_size]
selected_sample_size = max(sample_ties) # We want the largest number of samples (per cluster) in the case of a tie

# For features, use set operations to find common and combined sets of selected features
final_hdbscan_common_features = set.intersection(*all_features)
final_hdbscan_combined_features = set.union(*all_features)

In [ ]:
print(f' Final HDBSCAN common features = {final_hdbscan_common_features}')

In [ ]:
print(f' Final HDBSCAN combined features = {sorted(final_hdbscan_combined_features)}')

### Perform clustering with the reduced feature set

In [ ]:
# Add a test for whether there are any common results
if (final_hdbscan_common_features == set()) or (len(final_hdbscan_common_features) <= 5): # no common features OR not enough for processing
    print(' No common features - skipping ahead... ')
else:
    # Create a DataFrame for the best common features
    hdbscan_common_df = test_df.copy()
    hdbscan_reduced_common_features_df = hdbscan_common_df.iloc[:,sorted(final_hdbscan_common_features)]
    
    # Perform clustering on the final set of common features with the common cluster_size
    hdbscan_final_common_model = HDBSCAN(min_cluster_size=selected_cluster_size, min_samples=selected_sample_size, metric=most_common_hmetric, cluster_selection_method='eom', store_centers="medoid", allow_single_cluster=np.bool_(True), n_jobs=-1)
    hdbscan_final_common_labels = hdbscan_final_common_model.fit_predict(hdbscan_reduced_common_features_df)
    hdbscan_final_common_cluster_centers = hdbscan_final_common_model.medoids_
    
    # Create the dataframes for visualization
    hdbscan_final_viz_common_features_df = test_df.copy()
    hdbscan_final_reduced_common_features_df = kmedoids_final_viz_common_features_df.iloc[:,sorted(final_hdbscan_common_features)]
    hdbscan_final_reduced_common_features_df['HDBSCAN Clusters'] = hdbscan_final_common_labels
    
    hdbscan_final_COMMON_complete_features_df = test_df.copy()
    hdbscan_final_COMMON_complete_features_df['HDBSCAN Clusters'] = hdbscan_final_common_labels

# ------------------------------------------------------------------------------------------
# Create a DataFrame for the best combined features
hdbscan_combined_df = test_df.copy()
hdbscan_reduced_combined_features_df = hdbscan_combined_df.iloc[:,sorted(final_hdbscan_combined_features)]

# Perform clustering on the final set of combined features with the common cluster_size
hdbscan_final_combined_model = HDBSCAN(min_cluster_size=selected_cluster_size, min_samples=selected_sample_size, metric=most_common_hmetric, cluster_selection_method='eom', store_centers="medoid", allow_single_cluster=np.bool_(True), n_jobs=-1)
hdbscan_final_combined_labels = hdbscan_final_combined_model.fit_predict(hdbscan_reduced_combined_features_df)
hdbscan_final_combined_cluster_centers = hdbscan_final_combined_model.medoids_

# Create the dataframes for visualization
hdbscan_final_viz_combined_features_df = test_df.copy()
hdbscan_final_reduced_combined_features_df = hdbscan_final_viz_combined_features_df.iloc[:,sorted(final_hdbscan_combined_features)]
hdbscan_final_reduced_combined_features_df['HDBSCAN Clusters'] = hdbscan_final_combined_labels

hdbscan_final_COMBINED_complete_features_df = test_df.copy()
hdbscan_final_COMBINED_complete_features_df['HDBSCAN Clusters'] = hdbscan_final_combined_labels

### Generate reports to explore the clustering results (reduced feature set & complete feature set)

In [ ]:
# Start timing
start = time.perf_counter()

# Create YData reports to explore the HDBSCAN feature relationships
# DataFrames and configuration for the reports

if (final_hdbscan_common_features == set()) or (len(final_hdbscan_common_features) <= 5): # no common features OR not enough for processing
    print(' No common features ... ')
    now = str(time.time_ns()) # Create a timestamp for unique filename sets
    reports_info = [
        {
            'df': hdbscan_final_reduced_combined_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'HDBSCAN_Final_REDUCED_COMBINED-Features_Report-' + now + '.html'
        },
        {
            'df': hdbscan_final_COMBINED_complete_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'HDBSCAN_Final_COMPLETE_COMBINED-Features_Report-' + now + '.html'
        }
    ]
else:
    now = str(time.time_ns()) # Create a timestamp for unique filename sets
    reports_info = [
        {
            'df': hdbscan_final_reduced_common_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'HDBSCAN_Final_REDUCED_COMMON-Features_Report-' + now + '.html'
        },
        {
            'df': hdbscan_final_COMMON_complete_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'HDBSCAN_Final_COMPLETE_COMMON-Features_Report-' + now + '.html'
        },
        {
            'df': hdbscan_final_reduced_combined_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'HDBSCAN_Final_REDUCED_COMBINED-Features_Report-' + now + '.html'
        },
        {
            'df': hdbscan_final_COMBINED_complete_features_df,
            'config_file': 'config_ELR.yml',
            'output_file': 'HDBSCAN_Final_COMPLETE_COMBINED-Features_Report-' + now + '.html'
        }
    ]
# Use joblib to run the report generations in parallel
# n_jobs=-1 uses all available CPUs
Parallel(n_jobs=4)(delayed(generate_report)(
    info['df'], info['config_file'], info['output_file']) for info in reports_info)

# Stop timing
stop = time.perf_counter()

print(f" ^^^ Final HDBSCAN Clustering Report building in {stop - start:0.4f} seconds ^^^ ")

### Write Results to Project Database ###

In [ ]:
# load the config from the .env file
load_dotenv()
MONGODB_URI = os.environ['MONGODB_URI']

# Connect to the database engine
client = MongoClient(MONGODB_URI)

# connect to the project db
db = client['ExpectLifeRedux']

# get a reference to the data collection
#gov_data = db['Encoded_Gov_Data']

In [ ]:
# prefered method - use PyMongoArrow - write the dataframes to the database
write(db.Cluster_Unscaled_Complete, viz_df)
write(db.Cluster_Scaled_Complete, complete_df)
write(db.Cluster_PCA_Complete, complete_pca_df)
write(db.Cluster_KMedoids_Reduced_Features, kmedoids_final_reduced_features_df)
write(db.Cluster_KMedoids_Complete_Features, kmedoids_final_complete_features_df)
write(db.Cluster_HDBSCAN_Reduced_Features, hdbscan_final_reduced_features_df)
write(db.Cluster_HDBSCAN_Complete_Features, hdbscan_final_complete_features_df)

In [ ]:
#kmedoids_cluster_centers_df = pd.DataFrame(kmedoids_final_cluster_centers)
#write(db.Cluster_KMedoids_Centers, kmedoids_cluster_centers_df)

# Create the dataframe
#kmedoids_labels_df = pd.DataFrame(kmedoids_final_labels)
#write(db.Cluster_KMedoids_Labels, kmedoids_labels_df)

# Create the dataframe
#hdbscan_centers_df = pd.DataFrame(hdbscan_final_cluster_centers)
#write(db.Cluster_HDBSCAN_Centers, hdbscan_centers_df)

# Create the dataframe
#hdbscan_labels_df = pd.DataFrame(hdbscan_final_labels)
#write(db.Cluster_HDBSCAN_Labels, hdbscan_labels_df)





In [ ]:
kmedoids_best_features_df = pd.DataFrame()
kmedoids_best_features_df['Features'] = best_kmedoid_features
kmedoids_best_features_df

In [ ]:
hbdbscan_best_features_df = pd.DataFrame()
hbdbscan_best_features_df['Features'] = best_hdbscan_features
hbdbscan_best_features_df